In [ ]:
import boto3
import sagemaker.pytorch

# Access by job_name
job_name = '<TRAINING_JOB_NAME>'
pytorch_estimator = sagemaker.pytorch.estimator.PyTorch.attach(job_name, sagemaker_session=sagemaker.session.Session(boto_session=boto3.Session(region_name='ap-northeast-2')))

pytorch_model = sagemaker.pytorch.model.PyTorchModel(
    entry_point='suite_inference_single.py',
    name='suite-pytorch-inference-single',
    model_data=pytorch_estimator.model_data,
    role=pytorch_estimator.role,
    framework_version=pytorch_estimator.framework_version,
    py_version=pytorch_estimator.py_version,
)

predictor = pytorch_model.deploy(
    instance_type='ml.g4dn.xlarge',
    initial_instance_count=1,
)

In [ ]:
!python3 -m pip install --user spb-cli

In [ ]:
from urllib.request import urlopen
import sagemaker.pytorch
import spb.sdk

# Access by endpoint_name
endpoint_name = '<ENDPOINT_NAME>'
predictor = sagemaker.pytorch.model.PyTorchPredictor(endpoint_name)

predictor.serializer = sagemaker.serializers.IdentitySerializer('application/x-image')
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

req = {
    'auth': {
        'team_name': '',
        'access_key': '',
        'project_name': '',
    },
    'label_ids': [
        ''
    ],
}

print('Load Suite data...')
client = spb.sdk.Client(**req['auth'])
datas = [client.get_data(id=label_id) for label_id in req['label_ids']]
bin_images = [urlopen(data.get_image_url()).read() for data in datas]

print('Call SageMaker inference...')
outputs = [predictor.predict(bin_image) for bin_image in bin_images]

print('Upload predicted labels to Suite...')
for data, output in zip(datas, outputs):
    data.set_object_labels(output['objs'])
    data.update_data()

In [ ]:
import json, boto3
from urllib.request import urlopen
import spb.sdk

def predict(endpoint_name, body, content_type):                               
    client = boto3.client('sagemaker-runtime')                                      
    outputs = client.invoke_endpoint(                                               
        EndpointName=endpoint_name,
        ContentType=content_type,
        Accept='application/json',
        Body=body,
    )   
    return json.load(outputs['Body'])

req = {
    'auth': {
        'team_name': '',
        'access_key': '',
        'project_name': '',
    },
    'label_ids': [
        ''
    ],
}

endpoint_name = '<ENDPOINT_NAME>'

print('Load Suite data...')
client = spb.sdk.Client(**req['auth'])
datas = [client.get_data(id=label_id) for label_id in req['label_ids']]
bin_images = [urlopen(data.get_image_url()).read() for data in datas]

print('Call SageMaker inference...')
outputs = [predict(endpoint_name, bin_image, 'application/x-image') for bin_image in bin_images]

print('Upload predicted labels to Suite...')
for data, output in zip(datas, outputs):
    data.set_object_labels(output['objs'])
    data.update_data()

In [ ]:
import json, boto3
from urllib.request import urlopen
import spb.sdk

def predict(endpoint_name, body, content_type):
    client = boto3.client('sagemaker-runtime')
    outputs = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        Accept='application/json',
        Body=body,
    )   
    return json.load(outputs['Body'])

req = {
    'auth': {
        'team_name': '',
        'access_key': '',
        'project_name': '',
    },
    'label_ids': [
        ''
    ],
}

endpoint_name = '<ENDPOINT_NAME>'

print('Load Suite data...')
client = spb.sdk.Client(**req['auth'])
datas = [client.get_data(id=label_id) for label_id in req['label_ids']]
json_urls = [json.dumps({ 'image_url': data.get_image_url() }) for data in datas]

print('Call SageMaker inference...')
outputs = [predict(endpoint_name, json_url, 'application/json') for json_url in json_urls]

print('Upload predicted labels to Suite...')
for data, output in zip(datas, outputs):
    data.set_object_labels(output['objs'])
    data.update_data()